In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [5]:
df = pd.read_csv("../data/transactions.csv", nrows=1000000)
df.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [6]:
genders = pd.read_csv("../data/gender_train.csv")
genders.head()

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1


In [39]:
mccs = pd.read_csv("../data/tr_mcc_codes.csv", sep=';')
mccs.head()

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


In [8]:
tr_types = pd.read_csv("../data/tr_types.csv", sep=';')
tr_types.head()

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...


## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Если мы должны найти долю типов транзакций, которые содержат __POS__ или __ATM__ от всех уникальных типов транзакций в выборке `sample`:

In [21]:
# 1
sample = df['tr_type'].sample(1000)

In [22]:
unique_types_in_sample = sample.unique()

In [23]:
len(sample)

1000

### Ответ к заданию 1

In [24]:
# 2
tr_types[tr_types['tr_type'].isin(unique_types_in_sample)]['tr_description'] \
    .str.contains('POS|АТМ').value_counts(normalize=True)

True     0.642857
False    0.357143
Name: tr_description, dtype: float64

---
Если же мы должны найти долю транзакций в выборке `sample`, у которых в описании типа есть слова __POS__ или __ATM__ относительно всех 1000 транзакций в выборке, то:

In [27]:
types_contain_pos = tr_types[tr_types['tr_description'].str.contains('POS|АТМ')]['tr_type']

### Ответ к заданию 1

In [28]:
# 2
sample.isin(types_contain_pos).value_counts()  # нормализировать не имеет смысла, сумма = 1000

True     617
False    383
Name: tr_type, dtype: int64

###### Примечание: ответы будут меняться при каждом новом `sample`

###### Примечание от 24.09.2021: Правильная интерпретация задания - вторая

## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

### Ответы к заданию 2

In [119]:
# 1
df['tr_type'].value_counts()

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
         ...  
2446         4
4096         4
8146         3
1510         2
8100         2
Name: tr_type, Length: 72, dtype: int64

In [120]:
# 2
df['tr_type'].value_counts()[:10]

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
2370     49830
7010     28972
7030     26078
7071     15781
1100     15476
Name: tr_type, dtype: int64

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [121]:
money_sums = df.groupby('customer_id').sum()

In [122]:
# 1
money_sums[money_sums['amount'] == max(money_sums['amount'])]

,mcc_code,tr_type,amount
customer_id,,,
95481247,4227905,2282858,50508877.7


In [123]:
# 2
money_sums[money_sums['amount'] == min(money_sums['amount'])]

,mcc_code,tr_type,amount
customer_id,,,
54871257,15954177,7081907,-801026035.2


Если нам надо найти разницу между `amount` у самого "богатого" и `amount` у самого "бедного", то:

In [124]:
round(float(money_sums[money_sums['amount'] == min(money_sums['amount'])]['amount']) -
      float(money_sums[money_sums['amount'] == max(money_sums['amount'])]['amount']), 1)

-851534912.9

---
Если же нам надо найти разницу между всей прибылью и всей убылью у каждого из этих двух клиентов (`customer_id` 95481247 и `customer_id` 54871257), то:

In [125]:
richest = money_sums[money_sums['amount'] == max(money_sums['amount'])].index[0]

In [126]:
poorest = money_sums[money_sums['amount'] == min(money_sums['amount'])].index[0]

In [127]:
trs_of_richest = df[df['customer_id'] == richest]  # transactions of the richest customer

In [128]:
trs_of_poorest = df[df['customer_id'] == poorest]  # transactions of the poorest customer

In [129]:
# 3 (for customer_id = 95481247)
trs_of_richest[trs_of_richest['amount'] < 0]['amount'].sum() -\
trs_of_richest[trs_of_richest['amount'] > 0]['amount'].sum()

-122309451.36

In [130]:
# 3 (for customer_id = 54871257)
trs_of_poorest[trs_of_poorest['amount'] < 0]['amount'].sum() -\
trs_of_poorest[trs_of_poorest['amount'] > 0]['amount'].sum()

-1145257382.16

###### Примечание от 24.09.2021: Правильная интерпретация задания - вторая

## Задание 4
1. Найдите среднее арифметическое и медиану по всем типам транзакций из топ 10 из задания 2
2. Найдите среднее арифметическое и медиану по всем типам транзакций для клиентов из задания 3

### Ответ к заданию 4

In [131]:
top10_tr_types = df[df['tr_type'].isin(list(df['tr_type'].value_counts()[:10].index))]

In [132]:
# 1 (mean)
top10_tr_types.groupby('tr_type')['amount'].mean()

tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64

In [133]:
# 1 (median)
top10_tr_types.groupby('tr_type')['amount'].median()

tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64

In [134]:
# 2 (mean for the richest one)
trs_of_richest.groupby('tr_type')['amount'].mean()

tr_type
1010    -42570.722778
1030     -6446.932628
1100    -15508.020098
1110    -39140.697500
1200     -7244.027250
2010   -246489.256406
2330    -67377.470000
2370   -178690.461266
2371   -134754.950000
4041      -859.063333
4071     -1347.550000
6110     18349.130000
6200      6957.510000
7010    892241.084091
7020     51094.582500
7021    119595.015000
7030    119906.947778
7031    192025.799091
7070    193394.852128
7071    193634.035714
Name: amount, dtype: float64

In [135]:
# 2 (median for the richest one)
trs_of_richest.groupby('tr_type')['amount'].median()

tr_type
1010    -14901.655
1030     -2245.920
1100    -12105.490
1110    -27751.880
1200     -5832.645
2010   -179673.260
2330    -67377.470
2370    -67377.470
2371   -134754.950
4041      -892.750
4071     -1347.550
6110     18349.130
6200      7137.630
7010    673774.730
7020     59516.765
7021    126894.245
7030     94328.460
7031     97697.340
7070     85344.800
7071     58393.810
Name: amount, dtype: float64

In [136]:
# 2 (mean for the poorest one)
trs_of_poorest.groupby('tr_type')['amount'].mean()

tr_type
1010   -4.416506e+04
1030   -1.903366e+04
1100   -1.807962e+04
1110   -7.717905e+04
2010   -4.502637e+05
2011   -5.440107e+05
2020   -3.544629e+06
2110   -1.066810e+05
2330   -4.929340e+05
2331   -7.586204e+05
2370   -4.357385e+05
2371   -5.612563e+05
4010   -6.902448e+03
4051   -7.586206e+03
4071   -4.727519e+03
4110   -3.368877e+03
4500   -3.481169e+04
7010    1.135792e+06
7011    4.491830e+03
7021    5.545472e+05
7030    9.348624e+05
7031    3.671511e+05
7070    1.358207e+06
7071    3.660094e+05
Name: amount, dtype: float64

In [137]:
# 2 (median for the poorest one)
trs_of_poorest.groupby('tr_type')['amount'].median()

tr_type
1010     -19202.580
1030     -22459.160
1100     -14036.975
1110     -29982.980
2010    -336887.370
2011    -449183.150
2020   -1572141.040
2110    -106681.000
2330    -336887.370
2331    -505331.050
2370    -269509.890
2371    -363838.355
4010      -6737.750
4051      -5053.310
4071      -3638.385
4110      -2245.920
4500     -34811.695
7010     999432.520
7011       4491.830
7021     554547.150
7030     640085.995
7031     333518.490
7070     269509.890
7071     289049.360
Name: amount, dtype: float64

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [30]:
transactions = pd.merge(df, genders, how='left')
transactions = pd.merge(transactions, mccs, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

Кол-во строк совпадает

In [31]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

### Ответ к заданию 5

In [140]:
# 1
abs(transactions[transactions['amount'] < 0].groupby('gender')['amount'].mean()[0] -
    transactions[transactions['amount'] < 0].groupby('gender')['amount'].mean()[1])

32718.054920224655

32718.054920224655

In [141]:
# 2
abs(transactions[transactions['amount'] > 0].groupby('gender')['amount'].mean()[0] -
    transactions[transactions['amount'] > 0].groupby('gender')['amount'].mean()[1])

63366.57104801464

63366.57104801464

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.

In [32]:
transactions['mcc_code'].nunique()

184

In [33]:
gender_0 = {}
gender_1 = {}

### Ответ к заданию 7

In [34]:
# 1 (gender = 0)
transactions[transactions['gender'] == 0].groupby('mcc_code').sum()['amount']

mcc_code
742      -386207.68
1711     -674897.69
1799     -261626.73
2741      -20746.42
3000   -70435914.05
           ...     
8699     -521107.48
8999   -19421951.30
9222     -361143.24
9311     -372897.03
9399    -1481389.65
Name: amount, Length: 174, dtype: float64

In [35]:
# 1 (gender = 1)
transactions[transactions['gender'] == 1].groupby('mcc_code').sum()['amount']

mcc_code
742      -132277.93
1711     -206264.90
1731     -121542.67
1799    -1129257.68
2741     -221290.08
           ...     
8999   -28636743.64
9222     -988205.17
9311     -373271.22
9399    -1810820.67
9402       -4581.67
Name: amount, Length: 175, dtype: float64

In [36]:
g_0 = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)].groupby('mcc_code').sum()['amount']

In [37]:
g_1 = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)].groupby('mcc_code').sum()['amount']

In [38]:
# 2
(abs(g_1 - g_0)).sort_values(ascending=False)[:5]

mcc_code
6011    3.085862e+09
4829    2.499153e+09
6010    2.208580e+08
5541    1.554326e+08
5511    7.352877e+07
Name: amount, dtype: float64

In [40]:
(abs(g_1 - g_0)).sort_values(ascending=False)[:5].index

Int64Index([6011, 4829, 6010, 5541, 5511], dtype='int64', name='mcc_code')

In [41]:
mccs[mccs['mcc_code'].isin((abs(g_1 - g_0)).sort_values(ascending=False)[:5].index)]

,mcc_code,mcc_description
22,4829,Денежные переводы
62,5511,"Легковой и грузовой транспорт — продажа, серви..."
65,5541,Станции техобслуживания
123,6010,Финансовые институты — снятие наличности вручную
124,6011,Финансовые институты — снятие наличности автом...


А теперь проверим правильность полученного результата своими руками (с костылями, но зато убедимся)

In [42]:
for i in transactions['mcc_code'].unique():
    if i in g_0.index:
        gender_0[i] = g_0.loc[i]
    else:
        gender_0[i] = 0

In [43]:
for i in transactions['mcc_code'].unique():
    if i in g_1.index:
        gender_1[i] = g_1.loc[i]
    else:
        gender_1[i] = 0

In [44]:
differences = {}
for i in range(len(gender_0)):
    differences[list(gender_0.keys())[i]] = abs(list(gender_0.values())[i] -
                                                list(gender_1.values())[i])

In [45]:
sorted(differences.items(), key=lambda x: x[1], reverse=True)[:5]

[(6011, 3085861560.9300003),
 (4829, 2499153380.4900002),
 (6010, 220857973.82000002),
 (5541, 155432553.64999998),
 (5511, 73528766.9)]

6011,    3.085862e+09<br />
4829,    2.499153e+09<br />
6010,    2.208580e+08<br/>
5541,    1.554326e+08<br/>
5511,    7.352877e+07<br/>

Проверяем и видим, что обычная разность двух PandasSeries дала нам тот же результат

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте ночные поступления для мужчин и женщин. Ночные - это примерно 00-06 часов. 

In [46]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ


In [47]:
import re

In [48]:
lol = list(transactions['tr_datetime'].map(lambda x: re.search(r'\d{2}:', x).group()))

In [49]:
lol = list(map(lambda x: re.search(r'\d{2}', x).group(), lol))
lol = list(map(int, lol))

### Ответ к заданию 8

In [50]:
# 1
transactions['tr_hour'] = lol

In [51]:
# 1
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14


In [52]:
# 2 (gender = 0)
transactions[(transactions['gender'] == 0) & (transactions['tr_hour'] > 0) & (transactions['tr_hour'] < 6) &
             (transactions['amount'] > 0)].\
    groupby('customer_id').sum()['amount']

customer_id
117556       996063.66
293048      1825929.53
1747627      240312.99
1946162     1426156.55
2208164      429980.33
               ...    
96057039    2088701.67
98488344      94306.00
98804574     826946.18
98998782     156809.85
99683064      10106.63
Name: amount, Length: 151, dtype: float64

In [53]:
# 2 (gender = 1)
transactions[(transactions['gender'] == 1) & (transactions['tr_hour'] >= 0) & (transactions['tr_hour'] <= 6) &
             (transactions['amount'] > 0)].\
    groupby('customer_id').sum()['amount']

customer_id
189257       359346.52
361458      1174613.96
496551      1437386.10
1297829      183042.14
1521146       28803.87
               ...    
99267661    1684436.83
99302313     104435.08
99859464     336887.37
99864691     103312.13
99999680     233575.24
Name: amount, Length: 272, dtype: float64